In [1]:
import os, time, pickle
from lib import networks, utils
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
from torchvision import transforms
from torchvision import datasets

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
if torch.backends.cudnn.enabled:
    torch.backends.cudnn.benchmark = True
device

device(type='cuda')

### Hyper-parameters

In [31]:
#input channel for generator
in_ngc=3
#output channel for generator
out_ngc=3
#input channel for discriminator
in_ndc=6
#output channel for discriminator
out_ndc=1

batch_size=2
#number of filters in the first layer of generator
ngf=32
#number of filters in the first layer of discriminator
ndf=32
#the number of resnet block layer for generator
nb=9
#input size
input_size=128
train_epoch=100

#Discriminator learning rate, default=0.0002
lrD=0.0002
#Generator learning rate, default=0.0002
lrG=0.0002
#lambda for content loss
con_lambda=10
#beta1 for Adam optimizer
beta1=0.5
#beta2 for Adam optimizer
beta2=0.999

n_downsampling = 3

In [4]:
project_name = 'pix2pix_facades_1'
result_path = project_name+'_results'
data_name = 'data/facades'

# results save path
if not os.path.isdir(result_path):
    os.makedirs(result_path)
#ensure data folder exists
if not os.path.isdir(data_name):
    os.makedirs(data_name)
    print("data folder does not exist!!")


In [5]:
# data_loader
train_transform = transforms.Compose([
        transforms.Resize((input_size, 2*input_size)),
        transforms.ColorJitter(0.1,0.1,0.1,0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

train_loader = utils.data_load(data_name, 'train', train_transform, batch_size, shuffle=False, drop_last=True)
test_loader = utils.data_load(data_name, 'test', train_transform, batch_size, shuffle=False, drop_last=True)
# train_loader = torch.utils.data.DataLoader(datasets.ImageFolder(data_name, train_transform), batch_size=batch_size, shuffle=True, drop_last=True)

In [25]:
# network

# G = networks.generator(in_ngc, out_ngc, ngf, nb)
G = networks.UnetGenerator(in_ngc, out_ngc, 7, ngf)
D = networks.discriminator(in_ndc, out_ndc, ndf)
# D = networks.wgan_discriminator(in_ndc, ndf, input_size, n_downsampling)

G.to(device)
D.to(device)
G.train()
D.train();
# print('---------- Networks initialized -------------')
# utils.print_network(G)
# utils.print_network(D)
# print('-----------------------------------------------')

In [26]:
# loss
# GAN_loss = nn.BCELoss().to(device)
GAN_loss = nn.MSELoss().to(device)
L1_loss = nn.L1Loss().to(device)
# def D_loss_criterion(D_decision):
#     return D_decision.mean()

In [27]:
# Adam optimizer
G_optimizer = optim.Adam(G.parameters(), lr=lrG, betas=(beta1, beta2))
D_optimizer = optim.Adam(D.parameters(), lr=lrD, betas=(beta1, beta2))
G_scheduler = optim.lr_scheduler.MultiStepLR(optimizer=G_optimizer, milestones=[train_epoch // 2, train_epoch // 4 * 3], gamma=0.1)
D_scheduler = optim.lr_scheduler.MultiStepLR(optimizer=D_optimizer, milestones=[train_epoch // 2, train_epoch // 4 * 3], gamma=0.1)

In [28]:
train_hist = {}
train_hist['Disc_loss'] = []
train_hist['Gen_loss'] = []
train_hist['Con_loss'] = []
train_hist['per_epoch_time'] = []
train_hist['Gen_loss_one_epoch']=[]
train_hist['Disc_loss_one_epoch']=[]
train_hist['Con_loss_one_epoch']=[]

In [32]:
#starting_epoch is used to avoid overriding of the previously generated results
starting_epoch = 67

In [33]:
print('training start!')
start_time = time.time()
num_pool = 50
fake_pool = utils.ImagePool(num_pool)
# real = torch.ones(batch_size, 1, input_size // 4, input_size // 4).to(device)
# fake = torch.zeros(batch_size, 1, input_size // 4, input_size // 4).to(device)
for epoch in range(train_epoch):
    epoch_start_time = time.time()
    G.train()
    G_scheduler.step()
    D_scheduler.step()
    Disc_losses = []
    Gen_losses = []
    Con_losses = []
    for d, _ in train_loader:
        # x is the image at left, the source image
        # y is the image at right, the target image
        y = d[:, :, :, :input_size]
        x = d[:, :, :, input_size:]
        x, y, d = x.to(device), y.to(device), d.to(device)
        
        # train D
        for param in D.parameters():
            param.requires_grad = True
#             param.data.clamp_(-0.005,0.005)
        D_optimizer.zero_grad()

        real = torch.cat((y,x),1)
        D_real = D(real)
#         D_real_loss = D_loss_criterion(D_real)
        D_real_loss = GAN_loss(D_real, 1-torch.rand(D_real.size(),device = device)/10.0)
#         D_real_loss = GAN_loss(D_real, torch.ones(D_real.size(),device = device))

        with torch.no_grad():
            G_ = G(x)
        generated = torch.cat((G_,x), 1)
        generated = fake_pool.query(generated.detach())
        D_fake = D(generated)
#         D_fake_loss = D_loss_criterion(D_fake)
        D_fake_loss = GAN_loss(D_fake, torch.rand(D_fake.size(),device = device)/10.0)
#         D_fake_loss = GAN_loss(D_fake, torch.zeros(D_fake.size(),device = device))

        Disc_loss = 0.5 * (D_real_loss + D_fake_loss)
#         Disc_loss = D_fake_loss - D_real_loss
        Disc_losses.append(Disc_loss.item())
        train_hist['Disc_loss'].append(Disc_loss.item())
        Disc_loss.backward()
        D_optimizer.step()

        # train G
        for param in D.parameters():
            param.requires_grad = False
        G_optimizer.zero_grad()

        G_ = G(x)
        generated = torch.cat((G_,x), 1)
        D_fake = D(generated)
#         D_fake_loss = D_loss_criterion(D_fake)
        D_fake_loss = GAN_loss(D_fake, torch.ones(D_fake.size(),device = device))

        Con_loss = con_lambda * L1_loss(G_, y)

        Gen_loss = D_fake_loss + Con_loss
#         Gen_loss = -D_fake_loss + Con_loss

        Gen_losses.append(D_fake_loss.item())
        train_hist['Gen_loss'].append(D_fake_loss.item())
        Con_losses.append(Con_loss.item())
        train_hist['Con_loss'].append(Con_loss.item())

        Gen_loss.backward()
        G_optimizer.step()


    per_epoch_time = time.time() - epoch_start_time
    train_hist['per_epoch_time'].append(per_epoch_time)
    
    Gen_loss_avg = torch.mean(torch.FloatTensor(Gen_losses))
    Con_loss_avg = torch.mean(torch.FloatTensor(Con_losses))
    Disc_loss_avg =  torch.mean(torch.FloatTensor(Disc_losses))
    
    train_hist['Gen_loss_one_epoch'].append(Gen_loss_avg)
    train_hist['Disc_loss_one_epoch'].append(Disc_loss_avg)
    train_hist['Con_loss_one_epoch'].append(Con_loss_avg)
    
    print(
    '[%d/%d] - time: %.2f, Disc loss: %.3f, Gen loss: %.3f, Con loss: %.3f' % ((starting_epoch + epoch + 1), train_epoch, per_epoch_time, Disc_loss_avg, Gen_loss_avg, Con_loss_avg))

    with torch.no_grad():
        G.eval()
        for n, (d, _) in enumerate(train_loader):
            y = d[:, :, :, :input_size]
            x = d[:, :, :, input_size:]            
            x,y = x.to(device),y.to(device)
            G_recon = G(x)
            result = torch.cat((x[0], G_recon[0],y[0]), 2)
            path = os.path.join(result_path, str(starting_epoch+epoch+1) + '_epoch_' + project_name + '_train_' + str(n + 1) + '.png')
            plt.imsave(path, (result.cpu().numpy().transpose(1, 2, 0) + 1) / 2)
            if n == 2:
                break
        for n, (d, _) in enumerate(test_loader):
            y = d[:, :, :, :input_size]
            x = d[:, :, :, input_size:]            
            x,y = x.to(device),y.to(device)
            G_recon = G(x)
            result = torch.cat((x[0], G_recon[0],y[0]), 2)
            path = os.path.join(result_path, str(starting_epoch+epoch+1) + '_epoch_' + project_name + '_test_' + str(n + 1) + '.png')
            plt.imsave(path, (result.cpu().numpy().transpose(1, 2, 0) + 1) / 2)
            if n == 1:
                break
                
        torch.save(G.state_dict(), os.path.join(result_path, 'generator_latest.pkl'))
        torch.save(D.state_dict(), os.path.join(result_path, 'discriminator_latest.pkl'))
        with open(os.path.join(result_path,  'train_hist.pkl'), 'wb') as f:
            pickle.dump(train_hist, f)


training start!
[68/100] - time: 6.95, Disc loss: 0.074, Gen loss: 0.640, Con loss: 1.609
[69/100] - time: 6.84, Disc loss: 0.065, Gen loss: 0.612, Con loss: 1.593
[70/100] - time: 6.83, Disc loss: 0.064, Gen loss: 0.624, Con loss: 1.600
[71/100] - time: 7.34, Disc loss: 0.060, Gen loss: 0.640, Con loss: 1.580
[72/100] - time: 7.40, Disc loss: 0.063, Gen loss: 0.625, Con loss: 1.565
[73/100] - time: 6.85, Disc loss: 0.062, Gen loss: 0.628, Con loss: 1.562
[74/100] - time: 6.84, Disc loss: 0.071, Gen loss: 0.616, Con loss: 1.562
[75/100] - time: 6.83, Disc loss: 0.057, Gen loss: 0.632, Con loss: 1.564
[76/100] - time: 6.85, Disc loss: 0.067, Gen loss: 0.597, Con loss: 1.556
[77/100] - time: 6.82, Disc loss: 0.067, Gen loss: 0.626, Con loss: 1.558
[78/100] - time: 6.83, Disc loss: 0.058, Gen loss: 0.620, Con loss: 1.546
[79/100] - time: 6.81, Disc loss: 0.068, Gen loss: 0.617, Con loss: 1.528
[80/100] - time: 6.82, Disc loss: 0.067, Gen loss: 0.626, Con loss: 1.523
[81/100] - time: 6.83,

In [18]:
train_transform = transforms.Compose([
        transforms.Resize((input_size, 2*input_size)),
        transforms.ColorJitter(0.1,0.1,0.1,0.1),
        transforms.ToTensor(),
        transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

test_loader = torch.utils.data.DataLoader(datasets.ImageFolder('data/eye_test', train_transform), batch_size=batch_size, shuffle=True, drop_last=True)

test_result_path = 'pix2pix_eyes_test_results'

# results save path
if not os.path.isdir(test_result_path):
    os.makedirs(test_result_path)

with torch.no_grad():
    G.eval()
    for n, (d, _) in enumerate(test_loader):
#         x = d[:, :, :, :input_size]
#         y = d[:, :, :, input_size:]            
#         x = x.to(device)
#         y = y.to(device)
        d = d.to(device)
        G_recon = G(d)
        result = torch.cat((d[0], G_recon[0]), 2)
        path = os.path.join(test_result_path, str(starting_epoch+epoch+1) + '_epoch_' + project_name + '_test_' + str(n + 1) + '.png')
        plt.imsave(path, (result.cpu().numpy().transpose(1, 2, 0) + 1) / 2)